# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [48]:
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords


In [60]:
# your code here
tweet = pd.read_csv('../tweets.csv', encoding = "ISO-8859-1", header = None)
t_cols = ['target', 'id', 'date', 'flag', 'user', 'text']
tweet.columns = t_cols  #We include the column names
tweet.drop(tweet[tweet.target == 2].index, inplace=True) #Remove the "neutral" tweets rows (target = 2)
tweet = tweet.sample(frac=1) #Shuffle the rows to taque the sample
tweets = tweet.head(20000) #we keep the 20000 first rows

In [62]:
tweets.groupby('target').count()
# We can see that the rows are well distributeb between negative (0) and positive (4)

,id,date,flag,user,text
target,,,,,
0,9973,9973,9973,9973,9973
4,10027,10027,10027,10027,10027


In [63]:
tweets.dtypes

target     int64
id         int64
date      object
flag      object
user      object
text      object
dtype: object

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [64]:
# your code here
stop_words = set(stopwords.words('english')) 

In [65]:
def clean_up(s):
    
    no_url = re.sub(r'http\S+',' ', s)
    no_scha= re.sub(r'\W+', ' ', no_url)
    no_num = re.sub(r'\d+', ' ', no_scha)
    no_cap = no_num.lower()
    return no_cap

def tokenize(s):
    
    word_list = nltk.tokenize.word_tokenize(s)
    return word_list

def stem_and_lemmatize(l):
   
    stemming =[]
    for x in l:
        stemming.append(ps.stem(x))
    
    lemmatizing =[]
    for y in stemming:
        lemmatizing.append(lemmatizer.lemmatize(y))
    return lemmatizing

def remove_stopwords(l):
  
    list_sw = []
    for z in l:
        if z not in stop_words:
            list_sw.append(z)
    return list_sw

In [66]:
def processed_data(row):
    cleaned = clean_up(row)
    token = tokenize(cleaned)
    stem_lem = stem_and_lemmatize(token)
    text_processed = remove_stopwords(stem_lem)
    return text_processed
    

In [67]:
testing = tweets.iloc[0][5]


In [68]:
processed_data(testing)

['zachburnsid', 'want', 'ye']

In [69]:
tweets['text_processed'] = tweets.apply(lambda x: processed_data(x['text']), axis=1)

tweets.head()

<ipython-input-69-74d96471bf71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text_processed'] = tweets.apply(lambda x: processed_data(x['text']), axis=1)


,target,id,date,flag,user,text,text_processed
1191026,4,1983812940,Sun May 31 13:53:09 PDT 2009,NO_QUERY,attachingwings,"@zachburnside i want it, yes","[zachburnsid, want, ye]"
1496816,4,2070210277,Sun Jun 07 17:17:17 PDT 2009,NO_QUERY,JuliaFranco,"@Carolinaxoxo yess she even told us, A DURF","[carolinaxoxo, yes, even, told, u, durf]"
41579,0,1574105364,Tue Apr 21 04:16:02 PDT 2009,NO_QUERY,beccibeccibecci,Dans le school! Far to early for math class,"[dan, le, school, far, earli, math, class]"
54080,0,1680911679,Sat May 02 12:46:55 PDT 2009,NO_QUERY,houstonlimonet,@abednaderbc dont worry you will get them soon...,"[abednaderbc, dont, worri, get, soon, need, im..."
1540370,4,2180425315,Mon Jun 15 10:27:13 PDT 2009,NO_QUERY,jenna_jean,@RSD_Sufferer Thank you! lol,"[rsd_suffer, thank, lol]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [71]:
# your code here
from nltk.probability import FreqDist

In [72]:
freq_words = [item for sublist in tweets['text_processed'] for item in sublist]
freq_distribution = FreqDist(freq_words)
freq_distr_table = pd.DataFrame(freq_distribution, index=[0]).T.sort_values(0, ascending=False)
freq_distr_table = freq_distr_table.head(5000)
display(freq_distr_table)

,0
go,1702
get,1383
day,1365
wa,1302
thi,1235
...,...
transport,2
gabbysibajaost,2
por,2
griffin,2


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [14]:
# your code here
word_features = freq_distr_table.index
word_features

Index(['go', 'work', 'get', 'wa', 'thi', 'day', 'miss', 'like', 'got', 'want',
       ...
       'wag', 'craptast', 'propheci', 'fulfil', 'bella', 'bronchiti', 'gnarli',
       'bliss', 'boooooooo', 'cloudforc'],
      dtype='object', length=5000)

In [30]:
def find_features(tweet_text_list):
    words = set(tweet_text_list)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [31]:
print((find_features(tweets['text_processed'][0])))

{'go': False, 'work': False, 'get': False, 'wa': False, 'thi': False, 'day': True, 'miss': False, 'like': False, 'got': True, 'want': False, 'today': False, 'feel': False, 'back': False, 'time': False, 'still': False, 'realli': False, 'wish': False, 'good': False, 'one': False, 'im': False, 'quot': False, 'need': False, 'think': False, 'know': False, 'sad': False, 'night': False, 'u': False, 'ha': False, 'home': False, 'sleep': False, 'last': False, 'hope': False, 'sorri': False, 'love': False, 'na': False, 'make': False, 'bad': False, 'lol': False, 'oh': False, 'whi': False, 'much': False, 'see': False, 'watch': False, 'come': False, 'well': False, 'hate': False, 'onli': False, 'twitter': False, 'look': False, 'sick': False, 'amp': False, 'tomorrow': False, 'tri': False, 'morn': False, 'bed': False, 'though': False, 'new': False, 'would': False, 'week': False, 'veri': False, 'tonight': False, 'could': False, 'right': False, 'even': False, 'thing': False, 'hour': False, 'school': False

In [85]:
featuresets = []
for i in tweets.index:
    if tweets['target'][i] == 4:
        category = True
    else:
        category = False
    
    featuresets.append((find_features(tweets['text_processed'][i]), category))

In [87]:
print(featuresets[:1])

[({'go': False, 'work': False, 'get': False, 'wa': False, 'thi': False, 'day': False, 'miss': False, 'like': False, 'got': False, 'want': True, 'today': False, 'feel': False, 'back': False, 'time': False, 'still': False, 'realli': False, 'wish': False, 'good': False, 'one': False, 'im': False, 'quot': False, 'need': False, 'think': False, 'know': False, 'sad': False, 'night': False, 'u': False, 'ha': False, 'home': False, 'sleep': False, 'last': False, 'hope': False, 'sorri': False, 'love': False, 'na': False, 'make': False, 'bad': False, 'lol': False, 'oh': False, 'whi': False, 'much': False, 'see': False, 'watch': False, 'come': False, 'well': False, 'hate': False, 'onli': False, 'twitter': False, 'look': False, 'sick': False, 'amp': False, 'tomorrow': False, 'tri': False, 'morn': False, 'bed': False, 'though': False, 'new': False, 'would': False, 'week': False, 'veri': False, 'tonight': False, 'could': False, 'right': False, 'even': False, 'thing': False, 'hour': False, 'school': Fa

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [88]:
# your code here
training_set = featuresets[:16000]
testing_set = featuresets[16000:]

In [89]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [94]:
classifier.show_most_informative_features(10)

Most Informative Features
                     sad = True            False : True   =     22.0 : 1.0
                 headach = True            False : True   =     21.1 : 1.0
                   sadli = True            False : True   =     19.1 : 1.0
                  throat = True            False : True   =     17.1 : 1.0
                  welcom = True             True : False  =     15.3 : 1.0
                  bummer = True            False : True   =     14.4 : 1.0
                   smile = True             True : False  =     13.2 : 1.0
                   broke = True            False : True   =     12.8 : 1.0
                 horribl = True            False : True   =     12.7 : 1.0
                 stomach = True            False : True   =     12.4 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [95]:
# your code here
print(f"Naive Bayes Algorithm has accuracy of : {(nltk.classify.accuracy(classifier, testing_set))*100}")

Naive Bayes Algorithm has accuracy of : 72.625


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [20]:
# your code here


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [21]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [22]:
# your code here